## Naive Bayes - Mushroom Dataset

Goal is to predict the class of mushrooms, given some features of mushrooms. We will use Naive Bayes Model for this classification.

### Load the dataset

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("./mushrooms.csv")
df.head()

type cap_shape cap_surface cap_color bruises odor gill_attachment  \
0    p         x           s         n       t    p               f   
1    e         x           s         y       t    a               f   
2    e         b           s         w       t    l               f   
3    p         x           y         w       t    p               f   
4    e         x           s         g       f    n               f   

  gill_spacing gill_size gill_color   ...   stalk_surface_below_ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk_color_above_ring stalk_color_below_ring veil_type veil_color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring_number ring_type spore_print_color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

### Encode the Categorical Data into Numerical Data

In [3]:
le = LabelEncoder()

#Applies transformation on each columns
ds = df.apply(le.fit_transform)

In [4]:
ds.head()

type  cap_shape  cap_surface  cap_color  bruises  odor  gill_attachment  \
0     1          5            2          4        1     6                1   
1     0          5            2          9        1     0                1   
2     0          0            2          8        1     3                1   
3     1          5            3          8        1     6                1   
4     0          5            2          3        0     5                1   

   gill_spacing  gill_size  gill_color   ...     stalk_surface_below_ring  \
0             0          1           4   ...                            2   
1             0          0           4   ...                            2   
2             0          0           5   ...                            2   
3             0          1           5   ...                            2   
4             1          0           4   ...                            2   

   stalk_color_above_ring  stalk_color_below_ring  veil_type  veil_color  \
0                       7                       7          0           2   
1                       7                       7          0           2   
2                       7                       7          0           2   
3                       7                       7          0           2   
4                       7                       7          0           2   

   ring_number  ring_type  spore_print_color  population  habitat  
0            1          4                  2           3        5  
1            1          4                  3           2        1  
2            1          4                  3           2        3  
3            1          4                  2           3        5  
4            1          0                  3           0        1  

[5 rows x 23 columns]

In [5]:
data = ds.values
print(data.shape)
print(type(data))
print(data[:5,:])

data_x = data[:,1:]
data_y = data[:,0]

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


### Break data into train and test

In [6]:
x_train,x_test,y_train,y_test = train_test_split(data_x,data_y,test_size=0.2)

In [7]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [8]:
np.unique(y_train)

array([0, 1], dtype=int64)

### Building our classifier

In [9]:
a = np.array([0,5,5,1,1,1,0,1])
np.sum(a==5)

2

In [10]:
def prior_prob(y_train,label):
    
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train==label)
    
    return (class_examples)/float(total_examples)

In [11]:
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    
    x_filtered = x_train[y_train==label]
    numerator = np.sum(x_filtered[:,feature_col]==feature_val)
    denominator = np.sum(y_train==label)
    
    return numerator/float(denominator)

### Compute Posterior Probability for each test example and make predictions

In [12]:
def predict(x_train,y_train,xtest):
    
    classes = np.unique(y_train)
    n_features = x_train.shape[1]
    post_probs = []
    
    #Compute posterior for each class
    for label in classes:
        
        #Post_c = likelihood*prior
        likelihood = 1.0
        for f in range(n_features):
            cond = cond_prob(x_train,y_train,f,xtest[f],label)
            likelihood *= cond
        
        prior = prior_prob(y_train,label)
        post = likelihood*prior
        post_probs.append(post)
    
    pred = np.argmax(post_probs)
    return pred
        

In [13]:
output = predict(x_train,y_train,x_test[1])
print(output)
print(y_test[1])

1
1


In [14]:
def score(x_train,y_train,x_test,y_test):
    
    pred = []
    
    for i in range(x_test.shape[0]):
        pred_label = predict(x_train,y_train,x_test[i])
        pred.append(pred_label)
        
    pred = np.array(pred)
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    
    return accuracy

In [15]:
print(score(x_train,y_train,x_test,y_test))

0.9956923076923077
